In [1]:
%load_ext autoreload
%autoreload 2

import asyncio, copy, os, pickle, socket, sys, time
from functools import partial
from multiprocessing import Pool, Process
from pathlib import Path
from tqdm import tqdm

import torch
from torch import optim
from torch.utils.tensorboard import SummaryWriter

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../")))
from libs import agg, data, fl, log, nn, poison, resnet, sim
from cfgs.fedargs import *

In [2]:
# Save Logs To File (info | debug | warning | error | critical) [optional]
log.init("info")
#log.init("info", "poison.log")
#log.init("debug", "poison.log")

In [3]:
# Make changes as required by looking at the cfgs/fedargs
fedargs.agg_rule = agg.Rule.M_Krum
#fedargs.dataset = "cifar10" # for cifar-10
#fedargs.model = resnet.ResNet18() # for cifar10
fedargs.tb = SummaryWriter('../../out/runs/federated/M_Krum/mn-no-attack', comment="posion")
mal_clients = [c for c in range(20)]
FLTrust["is"] = False
FLTrust["proxy"]["is"] = False
label_flip_attack["is"] = False
label_flip_attack["func"] = poison.label_flip_next
set_lfa_labels()
backdoor_attack["is"] = False
layer_replacement_attack["is"] = False
cosine_attack["is"] = False
fang_attack["is"] = False
lie_attack["is"] = False
sota_attack["is"] = False
#dev_type: unit_vec
sota_attack["dev_type"] = "sign"
#sota_attack["dev_type"] = "std"
#func: sota_agnostic_min_max
sota_attack["func"] = poison.sota_agnostic_min_sum
#sota_attack["func"] = poison.sota_agr_tailored_trmean
sybil_attack["is"] = False

In [4]:
# Device settings
use_cuda = fedargs.cuda and torch.cuda.is_available()
torch.manual_seed(fedargs.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

In [5]:
# Prepare clients
host = socket.gethostname()
clients = [host + "(" + str(client + 1) + ")" for client in range(fedargs.num_clients)]

In [6]:
# Initialize Global and Client models
global_model = copy.deepcopy(fedargs.model)
# Load Data to clients
train_data, test_data = data.load_dataset(fedargs.dataset)

<h2>FLTrust</h2>

In [7]:
if FLTrust["is"]:
    train_data, FLTrust["data"] = data.random_split(train_data, FLTrust["ratio"])
    FLTrust["loader"] = torch.utils.data.DataLoader(FLTrust["data"], batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)
    
    if FLTrust["proxy"]["is"]:
        FLTrust["data"], FLTrust["proxy"]["data"] = data.random_split(FLTrust["data"], FLTrust["proxy"]["ratio"])
        FLTrust["loader"] = torch.utils.data.DataLoader(FLTrust["data"], batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)
        FLTrust["proxy"]["loader"] = torch.utils.data.DataLoader(FLTrust["proxy"]["data"], batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)

<h2>Prepare a backdoored loader for test</h2>

In [8]:
if backdoor_attack["is"]:
    train_data, backdoor_attack["data"] = data.random_split(train_data, backdoor_attack["ratio"])
    backdoor_attack["data"] = poison.insert_trojan(backdoor_attack["data"],
                                                   backdoor_attack["target_label"],
                                                   backdoor_attack["trojan_func"], 1)
    backdoor_attack["loader"] = torch.utils.data.DataLoader(backdoor_attack["data"], batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)

<h2>Load client's data</h2>

In [9]:
clients_data = data.split_data(train_data, clients)

<h2>Label Flip Attack</h2>

In [10]:
if label_flip_attack["is"]:
    for client in mal_clients:
        clients_data[clients[client]] = label_flip_attack["func"](clients_data[clients[client]],
                                                                  label_flip_attack["labels"],
                                                                  label_flip_attack["percent"])

<h2>Backdoor Attack</h2>

In [11]:
if backdoor_attack["is"]:
    for client in mal_clients:
        clients_data[clients[client]] = poison.insert_trojan(clients_data[clients[client]],
                                                             backdoor_attack["target_label"],
                                                             backdoor_attack["trojan_func"], 0.5)

In [12]:
client_train_loaders, _ = data.load_client_data(clients_data, fedargs.client_batch_size, None, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=fedargs.test_batch_size, shuffle=True, **kwargs)

client_details = {
        client: {"train_loader": client_train_loaders[client],
                 "model":  copy.deepcopy(global_model),
                 "model_update": None}
        for client in clients
    }

In [13]:
def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)

    return wrapped

@background
def process(client, epoch, model, train_loader, fedargs, device):
    # Train
    model_update, model, loss = fl.train_model(model, train_loader, 
                                            fedargs.learning_rate,
                                            fedargs.weight_decay,
                                            fedargs.local_rounds, device)

    log.jsondebug(loss, "Epoch {} of {} : Federated Training loss, Client {}".format(epoch, fedargs.epochs, client))
    log.modeldebug(model_update, "Epoch {} of {} : Client {} Update".format(epoch, fedargs.epochs, client))
    
    return model_update

In [14]:
import time
start_time = time.time()
    
# Federated Training
for _epoch in tqdm(range(fedargs.epochs)):

    epoch = _epoch + 1
    log.info("Federated Training Epoch {} of {}".format(epoch, fedargs.epochs))

    # Gloabal Model Update
    if epoch > 1:
        # For Tmean and FLTrust, not impacts others as of now
        avgargs = {"beta": len(mal_clients), 
                   "base_model_update": global_model_update if FLTrust["is"] else None,
                   "base_norm": True}
        
        # Average
        global_model = fl.federated_avg(client_model_updates, global_model, fedargs.agg_rule, **avgargs)
        log.modeldebug(global_model, "Epoch {} of {} : Server Update".format(epoch, fedargs.epochs))

        # Test
        global_test_output = fl.evaluate(global_model, test_loader, device, label_flip_attack["labels"])
        fedargs.tb.add_scalar("Gloabl Accuracy/", global_test_output["accuracy"], epoch)
        fedargs.tb.add_scalar("Gloabl Test Loss/", global_test_output["test_loss"], epoch)
        if "attack" in global_test_output:
            if "attack_success_rate" in global_test_output["attack"]:
                fedargs.tb.add_scalar("Attack Success Rate/", global_test_output["attack"]["attack_success_rate"], epoch)
            if "misclassification_rate" in global_test_output["attack"]:
                fedargs.tb.add_scalar("Misclassification Rate/", global_test_output["attack"]["misclassification_rate"], epoch)
        if backdoor_attack["is"]:
            backdoor_test_output = fl.backdoor_test(global_model, backdoor_attack["loader"], device,
                                                                backdoor_attack["target_label"])
            fedargs.tb.add_scalar("Backdoor Success Rate/", backdoor_test_output["accuracy"], epoch)
            log.jsoninfo(backdoor_test_output, "Backdoor Test Outut after Epoch {} of {}".format(epoch, fedargs.epochs))
        log.jsoninfo(global_test_output, "Global Test Outut after Epoch {} of {}".format(epoch, fedargs.epochs))
    
        # Update client models
        for client in clients:
            client_details[client]['model'] = copy.deepcopy(global_model)

    # Clients
    tasks = [process(client, epoch, client_details[client]['model'],
                     client_details[client]['train_loader'],
                     fedargs, device) for client in clients]
    try:
        updates = fedargs.loop.run_until_complete(asyncio.gather(*tasks))
    except KeyboardInterrupt as e:
        print("Caught keyboard interrupt. Canceling tasks...")
        tasks.cancel()
        fedargs.loop.run_forever()
        tasks.exception()

    for client, update in zip(clients, updates):
        client_details[client]['model_update'] = update
    client_model_updates = {client: details["model_update"] for client, details in client_details.items()}
    
    # Fang attack
    if fang_attack["is"]:
        client_model_updates = fang_attack["func"](client_model_updates, len(mal_clients))
        
    # LIE attack
    if lie_attack["is"]:
        client_model_updates = lie_attack["func"](client_model_updates, len(mal_clients))
        
    # SOTA attack
    if sota_attack["is"]:
        client_model_updates = sota_attack["func"](client_model_updates, len(mal_clients), sota_attack["dev_type"])    
    
    if FLTrust["is"]:
        global_model_update, _, _ = fl.train_model(global_model, FLTrust["loader"],
                                                fedargs.learning_rate,
                                                fedargs.weight_decay,
                                                fedargs.local_rounds, device)

        # For Attacks related to FLTrust
        base_model_update = global_model_update
        if FLTrust["proxy"]["is"]:
            base_model_update, _, _ = fl.train_model(global_model, FLTrust["proxy"]["loader"],
                                                  fedargs.learning_rate,
                                                  fedargs.weight_decay,
                                                  fedargs.local_rounds, device)
            
        if layer_replacement_attack["is"]:
            for client in mal_clients:
                client_details[clients[client]]['model_update'] = poison.layer_replacement_attack(base_model_update, 
                                                                                                  client_details[clients[client]]['model_update'],
                                                                                                  layer_replacement_attack["layers"])

        # For cosine attack, Malicious Clients
        if cosine_attack["is"]:
            b_arr, b_list = sim.get_net_arr(base_model_update)
            
            if epoch % cosine_attack["args"]["scale_epoch"] == 0:
                cosine_attack["args"]["scale_dot"] = cosine_attack["args"]["scale_dot_factor"] + cosine_attack["args"]["scale_dot"]
                cosine_attack["args"]["scale_norm"] = cosine_attack["args"]["scale_norm_factor"] * cosine_attack["args"]["scale_norm"]

            with Pool(len(mal_clients)) as p:
                func = partial(poison.model_poison_cosine_coord, b_arr, cosine_attack["args"])
                p_models = p.map(func, [sim.get_net_arr(client_details[clients[client]]['model_update'])[0]
                                        for client in mal_clients])
                p.close()
                p.join()


            for client, (p_arr, _) in zip(mal_clients, p_models):
                client_details[clients[client]]['model_update'] = sim.get_arr_net(client_details[clients[client]]['model_update'],
                                                                                  p_arr, b_list)
                
            #plot params changed for only one client
            fedargs.tb.add_scalar("Params Changed for Cosine Attack/", p_models[0][1], epoch)

        # For sybil attack, Malicious Clients
        if sybil_attack["is"]:
            for client in mal_clients:
                client_details[clients[client]]['model_update'] = base_model_update
                
        # again pair, as changed during attack
        client_model_updates = {client: details["model_update"] for client, details in client_details.items()}

print(time.time() - start_time)

  2%|▏         | 1/50 [01:11<58:41, 71.87s/it]


TypeError: 'builtin_function_or_method' object is not iterable

<h1> End </h1>